<a href="https://colab.research.google.com/github/bhar-gav/MDS-COLAB/blob/main/FID_entity_pyramid_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
print(os.getcwd())
folder = "/content/drive/My Drive/Colab Notebooks/MDS"
os.chdir(folder)
print(os.getcwd())

Mounted at /content/drive
/content
/content/drive/My Drive/Colab Notebooks/MDS


In [ ]:
!pip install datasets

In [3]:
from datasets import load_dataset

dataset = load_dataset("multi_news")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.src.cleaned:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

test.src.cleaned:   0%|          | 0.00/69.0M [00:00<?, ?B/s]

test.tgt:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [5]:
train=dataset["train"]
val=dataset["validation"]
test=dataset["test"]

print(train.info)


DatasetInfo(description='\nMulti-News, consists of news articles and human-written summaries\nof these articles from the site newser.com.\nEach summary is professionally written by editors and\nincludes links to the original articles cited.\n\nThere are two features:\n  - document: text of news articles seperated by special token "|||||".\n  - summary: news summary.\n', citation='\n@misc{alex2019multinews,\n    title={Multi-News: a Large-Scale Multi-Document Summarization Dataset and Abstractive Hierarchical Model},\n    author={Alexander R. Fabbri and Irene Li and Tianwei She and Suyi Li and Dragomir R. Radev},\n    year={2019},\n    eprint={1906.01749},\n    archivePrefix={arXiv},\n    primaryClass={cs.CL}\n}\n', homepage='https://github.com/Alex-Fabbri/Multi-News', license='For non-commercial research and educational purposes only', features={'document': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=SupervisedKeysDat

**Preprocess**

In [6]:
import pandas as pd


data = []

for entry in train:
    all_docs = entry["document"].split("|||||")[:-1]
    for i, doc in enumerate(all_docs):
        doc = doc.replace("\n", " ")
        doc = " ".join(doc.split())
        all_docs[i] = doc

    tgt = entry["summary"]

    data.append({"documents": all_docs, "summary": tgt})

df = pd.DataFrame(data)
print(df.head())


                                           documents  \
0  [National Archives Yes, it’s that time again, ...   
1  [LOS ANGELES (AP) — In her first interview sin...   
2  [GAITHERSBURG, Md. (AP) — A small, private jet...   
3  [Tucker Carlson Exposes His Own Sexism on Twit...   
4  [A man accused of removing another man's testi...   

                                             summary  
0  – The unemployment rate dropped to 8.2% last m...  
1  – Shelly Sterling plans "eventually" to divorc...  
2  – A twin-engine Embraer jet that the FAA descr...  
3  – Tucker Carlson is in deep doodoo with conser...  
4  – What are the three most horrifying words in ...  


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44972 entries, 0 to 44971
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   documents  44972 non-null  object
 1   summary    44972 non-null  object
dtypes: object(2)
memory usage: 702.8+ KB


#**Entity etraction**

---



In [ ]:
!pip install spacy

**Entity Extraction**

In [ ]:
import spacy
import pandas as pd


nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    return [ent.text.lower() for ent in doc.ents]  # Lowercase the entities for case-insensitive matching

def truncate_document(doc_text, max_length=1000000):
    if len(doc_text) > max_length:
        print(f"Document truncated from {len(doc_text)} to {max_length} characters.")
        return doc_text[:max_length-1]
    return doc_text

#
def extract_entities_from_documents(docs):
    all_entities = []
    for doc in docs:
        truncated_doc = truncate_document(doc)
        doc_entities = extract_entities(truncated_doc)
        all_entities.append(doc_entities)
    return all_entities



df['entities_docs'] = df['documents'].apply(lambda docs: extract_entities_from_documents(docs))

df.head()


Document truncated from 2894251 to 1000000 characters.


,documents,summary,entities_docs
0,"[National Archives Yes, it’s that time again, ...",– The unemployment rate dropped to 8.2% last m...,"[[national archives yes, the first friday of t..."
1,[LOS ANGELES (AP) — In her first interview sin...,"– Shelly Sterling plans ""eventually"" to divorc...","[[los angeles, ap, first, nba, shelly sterling..."
2,"[GAITHERSBURG, Md. (AP) — A small, private jet...",– A twin-engine Embraer jet that the FAA descr...,"[[gaithersburg, md., ap, maryland, montgomery ..."
3,[Tucker Carlson Exposes His Own Sexism on Twit...,– Tucker Carlson is in deep doodoo with conser...,"[[tucker carlson exposes, twitter, tucker carl..."
4,[A man accused of removing another man's testi...,– What are the three most horrifying words in ...,"[[port macquarie, allan george matthews, 57, p..."


In [ ]:
df.head()

,documents,summary,entities_docs
0,"[National Archives Yes, it’s that time again, ...",– The unemployment rate dropped to 8.2% last m...,"[[national archives yes, the first friday of t..."
1,[LOS ANGELES (AP) — In her first interview sin...,"– Shelly Sterling plans ""eventually"" to divorc...","[[los angeles, ap, first, nba, shelly sterling..."
2,"[GAITHERSBURG, Md. (AP) — A small, private jet...",– A twin-engine Embraer jet that the FAA descr...,"[[gaithersburg, md., ap, maryland, montgomery ..."
3,[Tucker Carlson Exposes His Own Sexism on Twit...,– Tucker Carlson is in deep doodoo with conser...,"[[tucker carlson exposes, twitter, tucker carl..."
4,[A man accused of removing another man's testi...,– What are the three most horrifying words in ...,"[[port macquarie, allan george matthews, 57, p..."


In [ ]:
df.to_csv("train_fid.csv", index=False)

#**2. Entity pyramid construction**
---

**> Salient sentence selection**

In [2]:
import pandas as pd
df=pd.read_csv("train_fid.csv")
df.head()

,documents,summary,entities_docs
0,"['National Archives Yes, it’s that time again,...",– The unemployment rate dropped to 8.2% last m...,"[['national archives yes', 'the first friday o..."
1,['LOS ANGELES (AP) — In her first interview si...,"– Shelly Sterling plans ""eventually"" to divorc...","[['los angeles', 'ap', 'first', 'nba', 'shelly..."
2,"[""GAITHERSBURG, Md. (AP) — A small, private je...",– A twin-engine Embraer jet that the FAA descr...,"[['gaithersburg', 'md.', 'ap', 'maryland', 'mo..."
3,['Tucker Carlson Exposes His Own Sexism on Twi...,– Tucker Carlson is in deep doodoo with conser...,"[['tucker carlson exposes', 'twitter', 'tucker..."
4,"[""A man accused of removing another man's test...",– What are the three most horrifying words in ...,"[['port macquarie', 'allan george matthews', '..."


#sentence selection

In [3]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a4b217f381b901d32e5234dbc215257b853338a33292aa102bc70bd853e8a225
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [13]:
import pandas as pd
import spacy
from collections import defaultdict
from rouge_score import rouge_scorer

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Sentence splitter using spaCy
def spacy_sent_tokenize(text):
    return [sent.text.strip() for sent in nlp(text).sents]

# Core algorithm (unchanged, except sentence splitting source)
def entity_pyramid_sentence_selection_rouge(documents, entity_doc_per_doc, m):
    entity_salience = defaultdict(int)
    for doc_entities in entity_doc_per_doc:
        for entity in set(doc_entities):
            entity_salience[entity] += 1  # salience = doc frequency

    frequent_entities = [e for e in entity_salience if entity_salience[e] > 1]
    sorted_entities = sorted(frequent_entities, key=lambda e: entity_salience[e], reverse=True)

    selected = []
    seen_sentences = set()
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

    doc_sentences = [spacy_sent_tokenize(doc) for doc in documents]

    for entity in sorted_entities:
        sent_candidates = []
        for doc_idx, doc in enumerate(doc_sentences):
            for sent in doc:
                if entity in sent:
                    sent_candidates.append((sent, doc_idx))

        if not sent_candidates:
            continue

        def score_sent(sent, doc_idx):
            cluster_text = " ".join(
                s for i, doc in enumerate(doc_sentences) if i != doc_idx for s in doc
            )
            rouge = scorer.score(sent, cluster_text)
            return rouge['rouge1'].fmeasure

        cur_sent = max(sent_candidates, key=lambda tup: score_sent(tup[0], tup[1]))[0]

        if cur_sent not in seen_sentences:
            selected.append(cur_sent)
            seen_sentences.add(cur_sent)

        if len(selected) == m:
            break

    return selected

# Wrapper for DataFrame row
def process_row(row, m=5):
    entity_doc_per_doc = []
    for doc in row['documents']:
        matched_entities = [entity for entity in row['entities_docs'] if entity in doc]
        entity_doc_per_doc.append(matched_entities)

    return entity_pyramid_sentence_selection_rouge(
        documents=row['documents'],
        entity_doc_per_doc=entity_doc_per_doc,
        m=m
    )



In [15]:
test_output = df.head(5).apply(lambda row: process_row(row, m=5), axis=1)
print(test_output)


0    [e, t, o, n, r]
1    [e, t, n, i, o]
2    [e, a, n, o, t]
3    [e, t, o, a, i]
4    [e, t, a, o, i]
dtype: object


In [ ]:
 df['selected_sentences'] = df.apply(lambda row: process_row(row, m=5), axis=1)



---



---
#**Model**



xsum
```
DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})
```



In [ ]:
!pip install transformers datasets torch rouge-score


In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [ ]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
xsum = xsum.map(preprocess_function, batched=True)
xsum

In [ ]:
xsum=xsum.remove_columns(['document', 'summary','id'])
xsum

In [ ]:
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

model=T5ForConditionalGeneration.from_pretrained('t5-small')


In [ ]:
# training arguments
training_arguments = Seq2SeqTrainingArguments(
            output_dir='./results',
            evaluation_strategy='epoch',

            save_strategy='epoch',  #save model state after each epoch (space consuming)
            #save_strategy='no',  #1. use to save model only once after finishing training (comment out above line in case)

            logging_dir='./logs',
            learning_rate=2e-5,
            per_device_train_batch_size=2,
            per_device_eval_batch_size=2,
            weight_decay=0.01,
            save_total_limit=4,
            num_train_epochs=2,
            # remove_unused_columns=False,
            fp16=True,
            )


trainer = Seq2SeqTrainer(
    model=model,
    args=training_arguments,
    train_dataset=xsum['train'],
    eval_dataset=xsum['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator
    )


In [ ]:
trainer.train()

#trainer.save_model()  # 2.save model after completing training (use if (1.) is followed)

In [ ]:
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
import torch

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def generate_summaries(test_dataset, model, tokenizer, batch_size=4, max_output_length=20):
    model.eval()
    all_summaries = []

    dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=data_collator)

    for batch in dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)

        summary_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_output_length,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True
        )


        summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
        all_summaries.extend(summaries)

    return all_summaries

generated_summaries = generate_summaries(xsum['test'], model, tokenizer)

for i in range(5):
    print(f"Generated Summary {i+1}: {generated_summaries[i]}")


In [ ]:
!pip install evaluate


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

def decode_labels(labels, tokenizer):
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return decoded_labels

generated_summaries = generate_summaries(xsum['test'], model, tokenizer)
reference_summaries = decode_labels(xsum['test']['labels'], tokenizer)


def calculate_rouge(generated_summaries, reference_summaries):
    results = rouge.compute(predictions=generated_summaries, references=reference_summaries, rouge_types=["rouge1", "rouge2", "rougeL", "rouge4"])
    return results

rouge_results = calculate_rouge(generated_summaries, reference_summaries)


print("ROUGE Scores:")
print(rouge_results)